In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
from scipy.spatial.distance import mahalanobis
import os
import sys
from PIL import Image
sys.path.append('../models')
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
##########################################
######### Configure Metric Setting ####### 
##########################################
batch_size = 64
weight_path = './weights/celebAHQ_all_resnet18_v3.pt'
model = torch.load(weight_path)
criterion = nn.CrossEntropyLoss()
dataset_path = "/media/data2/data/CelebAMask-HQ/CelebA-HQ-img/"
annotation_path = "/media/data2/data/CelebAMask-HQ/CelebAMask-HQ-attribute-anno.txt"
data = 'CelebA'
##########################################
##########################################

In [4]:
class CelebA(Dataset):
    def __init__(self, root, annotation_path, transform=None):
        self.transform = transform
        self.root_dir = root
        self.classes = [
            "Female, Not smiling, Young", "Female, Not smiling, Old", 
            "Female, Smiling, Young", "Female, Smiling, Old", 
            "Male, Not smiling, Young", "Male, Not smiling, Old", 
            "Male, Smiling, Young", "Male, Smiling, Old",
        ]
        
        self.image_paths = []
        self.image_labels = []

        anno_file = open(annotation_path, "r")
        num = int(anno_file.readline())
    
        line = anno_file.readline()
        attributes = list(line[:-1].split(' '))
        
        sex_index = attributes.index('Male') + 2
        smile_index = attributes.index('Smiling') + 2
        young_index = attributes.index('Young') + 2
        for _ in range(num):
            line = anno_file.readline()
            record = list(line[:-1].split(' '))
            self.image_paths.append(record[0])
            if record[sex_index] == "1":
                if record[smile_index] == "1":
                    if record[young_index] == "1":
                        self.image_labels.append(6)
                    else:
                        self.image_labels.append(7)
                else:
                    if record[young_index] == "1":
                        self.image_labels.append(4)
                    else:
                        self.image_labels.append(5)
            else:
                if record[smile_index] == "1":
                    if record[young_index] == "1":
                        self.image_labels.append(2)
                    else:
                        self.image_labels.append(3)
                else:
                    if record[young_index] == "1":
                        self.image_labels.append(0)
                    else:
                        self.image_labels.append(1)
        anno_file.close()
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_file_path = os.path.join(self.root_dir, self.image_paths[index])
        image = Image.open(image_file_path)

        label = self.image_labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [5]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) 
if data == 'STL-10':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
elif data == 'Cifar-10':
    mean, std = (0.491, 0.482, 0.447), (0.247, 0.243, 0.262)
elif data == 'CelebA':
    mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

model.to(device)
model.eval()

dataset =  CelebA(root=dataset_path, annotation_path=annotation_path, transform=trans)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=False, )

classes = dataset.classes

In [6]:
latent_vectors = [[] for _ in range(len(classes))]

model.eval()
with torch.no_grad():
    for data in dataloader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))

In [7]:
print("#####################################")
print("### Calculating Cosine Similarity ###")
print("#####################################")


for i in range(len(latent_vectors)):
    cssm = cosine_similarity(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(cssm):.4f}, min: {np.min(cssm):.4f}, mean: {np.mean(cssm):.4f}, var: {np.var(cssm):.4f}")


#####################################
### Calculating Cosine Similarity ###
#####################################
class: Female, Not smiling, Young, max: 1.0000, min: 0.5501, mean: 0.9953, var: 0.0001
class: Female, Not smiling, Old, max: 1.0000, min: 0.0490, mean: 0.9176, var: 0.0132
class: Female, Smiling, Young, max: 1.0000, min: 0.4102, mean: 0.9963, var: 0.0001
class: Female, Smiling, Old, max: 1.0000, min: 0.2921, mean: 0.9816, var: 0.0018
class: Male, Not smiling, Young, max: 1.0000, min: 0.7036, mean: 0.9964, var: 0.0001
class: Male, Not smiling, Old, max: 1.0000, min: 0.6150, mean: 0.9928, var: 0.0002
class: Male, Smiling, Young, max: 1.0000, min: 0.2053, mean: 0.9824, var: 0.0016
class: Male, Smiling, Old, max: 1.0000, min: 0.4036, mean: 0.9712, var: 0.0027


In [8]:
print("#####################################")
print("## Calculating Euclidean Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('euclidean')
for i in range(len(latent_vectors)):
    euclidean = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(euclidean):.4f}, min: {np.min(euclidean):.4f}, mean: {np.mean(euclidean):.4f}, var: {np.var(euclidean):.4f}")

#####################################
## Calculating Euclidean Distance ###
#####################################
class: Female, Not smiling, Young, max: 25.7890, min: 0.0000, mean: 4.2349, var: 8.5840
class: Female, Not smiling, Old, max: 8.2642, min: 0.0000, mean: 1.3351, var: 0.8081
class: Female, Smiling, Young, max: 28.6131, min: 0.0000, mean: 4.5219, var: 10.1270
class: Female, Smiling, Old, max: 9.0268, min: 0.0000, mean: 1.9927, var: 1.6364
class: Male, Not smiling, Young, max: 19.8191, min: 0.0000, mean: 3.1529, var: 4.9191
class: Male, Not smiling, Old, max: 16.0452, min: 0.0000, mean: 2.4795, var: 2.8036
class: Male, Smiling, Young, max: 19.9432, min: 0.0000, mean: 2.7509, var: 3.2489
class: Male, Smiling, Old, max: 12.9015, min: 0.0000, mean: 2.5921, var: 2.5493


In [9]:
print("#####################################")
print("## Calculating Manhattan Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('manhattan')
for i in range(len(latent_vectors)):
    manhattan = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(manhattan):.4f}, min: {np.min(manhattan):.4f}, mean: {np.mean(manhattan):.4f}, var: {np.var(manhattan):.4f}")

#####################################
## Calculating Manhattan Distance ###
#####################################
class: Female, Not smiling, Young, max: 67.3792, min: 0.0000, mean: 10.9606, var: 58.5363
class: Female, Not smiling, Old, max: 20.6594, min: 0.0000, mean: 3.3106, var: 5.1956
class: Female, Smiling, Young, max: 74.6292, min: 0.0000, mean: 11.7793, var: 70.0421
class: Female, Smiling, Old, max: 24.3042, min: 0.0000, mean: 5.1864, var: 11.7983
class: Male, Not smiling, Young, max: 49.7612, min: 0.0000, mean: 8.0273, var: 32.3957
class: Male, Not smiling, Old, max: 40.8285, min: 0.0000, mean: 6.4328, var: 19.6239
class: Male, Smiling, Young, max: 51.2475, min: 0.0000, mean: 6.9456, var: 21.3290
class: Male, Smiling, Old, max: 31.8374, min: 0.0000, mean: 6.5569, var: 16.4241


In [10]:
print("#####################################")
print("## Calculating Chebyshev Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('chebyshev')
for i in range(len(latent_vectors)):
    chebyshev = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(chebyshev):.4f}, min: {np.min(chebyshev):.4f}, mean: {np.mean(chebyshev):.4f}, var: {np.var(chebyshev):.4f}")

#####################################
## Calculating Chebyshev Distance ###
#####################################
class: Female, Not smiling, Young, max: 12.5287, min: 0.0000, mean: 2.0785, var: 1.9138
class: Female, Not smiling, Old, max: 4.4376, min: 0.0000, mean: 0.7436, var: 0.2208
class: Female, Smiling, Young, max: 13.5444, min: 0.0000, mean: 2.1275, var: 2.0391
class: Female, Smiling, Old, max: 4.7805, min: 0.0000, mean: 1.0498, var: 0.4107
class: Male, Not smiling, Young, max: 10.3258, min: 0.0000, mean: 1.6851, var: 1.3822
class: Male, Not smiling, Old, max: 7.9983, min: 0.0000, mean: 1.2338, var: 0.6041
class: Male, Smiling, Young, max: 10.8997, min: 0.0000, mean: 1.5140, var: 0.9371
class: Male, Smiling, Old, max: 6.3987, min: 0.0000, mean: 1.3443, var: 0.6456
